In [105]:
import win32com.client
from datetime import datetime, timedelta
from openpyxl import Workbook, load_workbook
from datetime import date

#data atual no formato dia.mes.ano
dia = date.today()
dia = dia.strftime("%d.%m.%Y")

nome_arquivo = "Gerenciamento POSICAO " + dia + ".xlsx"

#Workbook
wb = Workbook()

#Worksheet
ws = wb.active

ws.cell(row=1, column=1, value="Tipo")
ws.cell(row=1, column=2, value="DataMovimentacao")
ws.cell(row=1, column=3, value="Unidade")
ws.cell(row=1, column=4, value="Cliente")
ws.cell(row=1, column=5, value="Motivo")
ws.cell(row=1, column=6, value="Estado")

wb.save(filename = nome_arquivo)

celulas = []


#função que preenche uma lista com as celulas a serem adicionadas na planilha
def adicionarNaPlanilha(worksheet, tabelaEntrada, tabelaRetorno, GERAT):
    #Entradas
    celula = ""
    if(tabelaEntrada != ""):
        #preenchimento da lista
        for i in range(len(tabelaEntrada)):
            if(tabelaEntrada[i] != "\t" and tabelaEntrada[i] != "\n"):
                celula += tabelaEntrada[i]
            else:
                if(celula != " \r"):
                    celulas.append(celula.strip())
                celula = ""
        
        #dados são colocados na tabela a partir da primeira linha disponível
        maxRow = ws.max_row
        for i in range(int(len(celulas)/4)):
            ws.cell(row=i+maxRow+1, column=1, value="Entrada")
            for j in range(4):
                ws.cell(row=i+maxRow+1, column=j+2, value=celulas[i*4+j])
            ws.cell(row=i+maxRow+1, column=6, value=GERAT)
    
    #Saídas
    celula = ""
    celulas.clear()
    if(tabelaRetorno != ""):
        #preenchimento da lista
        for i in range(len(tabelaRetorno)):
            if(tabelaRetorno[i] != "\t" and tabelaRetorno[i] != "\n"):
                celula += tabelaRetorno[i]
            else:
                if(celula != " \r"):
                    celulas.append(celula.strip())
                celula = ""
        
        #dados são colocados na tabela a partir da primeira linha disponível
        maxRow = ws.max_row
        for i in range(int(len(celulas)/3)):
            ws.cell(row=i+maxRow+1, column=1, value="Retorno")
            for j in range(3):
                ws.cell(row=i+maxRow+1, column=j+2, value=celulas[i*3+j])
            ws.cell(row=i+maxRow+1, column=5, value="")
            ws.cell(row=i+maxRow+1, column=6, value=GERAT)
    celulas.clear()
    wb.save(filename = nome_arquivo)
    
    
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

folder = outlook.GetDefaultFolder(6).Folders("Gerenciamento")

messages = folder.Items

date = datetime.now() - timedelta(days=1)
date = date.strftime('%m/%d/%Y %H:%M %p')
messages = messages.Restrict("[ReceivedTime] >= '" + date + "'")


for message in messages:
    
    #substring referente ao nome do estado
    GERAT = message.body[message.body.find("Ativos-")+7:message.body.find(", conforme previsto no")]
    
    
    #tabela de migrações de entrada, caso exista
    tabelaEntrada = ""
    if(message.body.find("Motivo")):
        tabelaEntrada = message.body[message.body.find("Motivo")+10:message.body.find("Além de dar ciência ")]
        tabelaEntrada = tabelaEntrada[:tabelaEntrada.find("Total:")-1]
    
    
    #tabela de migrações de retorno, caso exista
    tabelaRetorno = ""
    if(message.body.find("Retorno")):
        tabelaRetorno = message.body[message.body.find("Retorno"):message.body.find("Ambiente de Suporte à Rede de Agências")]
        tabelaRetorno = tabelaRetorno[tabelaRetorno.find("Cliente")+11:tabelaRetorno.find("Total:")-1]
     
    adicionarNaPlanilha(ws, tabelaEntrada, tabelaRetorno, GERAT)